In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import fastai
from fastai.vision import *
from fastai.callback.tracker import SaveModelCallback
import os
#from sklearn.model_selection import KFold
from torch.optim import RAdam
#from csvlogger import *
from torch.nn import Mish
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score,confusion_matrix
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

fastai.__version__

'2.7.9'

In [2]:
sz = 128
bs = 4
nfolds = 4
SEED = 2022
N = 12 #number of tiles per image

## Data

In [3]:
#|export
from ml.nb_03_dataset import *

In [43]:
from fastai.data.core import DataLoaders

In [50]:
from fastai.vision.all import *

In [37]:
import seaborn as sns
import numpy as np

In [5]:
df = pd.read_pickle("data/df_all_2022_10_06.pkl")
df.shape

(1180, 95)

In [6]:
df.loc[df["G"]=="2", "G"] = 2
df = df.loc[df["G"]!="1,2"]
print(df.shape)

(1177, 95)


In [7]:
df.G.value_counts()

2    982
1    126
3     69
Name: G, dtype: int64

In [ ]:
df["label"]= df["G"]

In [38]:
def split_df(df, n=0.8 # percentage training
            ):
    df = df.sample(frac=1)
    msk = np.random.rand(len(df)) < n
    return df[msk], df[~msk]

In [39]:
df_train, df_test = split_df(df)

In [40]:
df_train.shape, df_test.shape

((941, 95), (236, 95))

In [42]:
ds_train = PatchedDataSet(p_outx, df_train, "G", "stime_10y_dom", N, mean_img, std_img)
ds_test = PatchedDataSet(p_outx, df_test, "G", "stime_10y_dom", N, mean_img, std_img)

In [41]:
#ds = PatchedDataSet(p_outy, df, "G", "stime_10y_dom", N, mean_mask, std_mask)
#i = 1
#arr = ds[i][0].cpu().detach().numpy()
#arr.shape
#img = arr[0,0,:]
#img.shape
#sns.heatmap(img , cmap = 'coolwarm', xticklabels=False, yticklabels=False);
#sns.heatmap(img , cmap = 'coolwarm', xticklabels=False, yticklabels=False);

In [61]:
dls = DataLoaders.from_dsets(ds_train, ds_test)

In [62]:
dls = dls.cuda()

In [67]:
b = dls.one_batch()

In [69]:
print(len(b), b[0].size(), b[1].size())

2 torch.Size([64, 12, 3, 128, 128]) torch.Size([64])


In [55]:
dblock = DataBlock(
  blocks=(ImageBlock, CategoryBlock),      # one image input and one categorical targets
  getters=[ColReader('path'),   # image input
           ColReader('label')],                    # label 1
  splitter=RandomSplitter(valid_pct=0.2, seed=42),
    item_tfms= Normalize(mean_img[0], std_img[0])
           )
dls_x = dblock.dataloaders(df_x, bs=64)   # Create the dataloaders

AttributeError: 'Tensor' object has no attribute 'show'